# 项目7-元学习

## 友情提示
同学们可以前往课程作业区先行动手尝试！！！

## 项目描述

通过MAML算法实现regression。

## 数据集介绍

本部分内容主要是通过随机的一些数据学习，并实现拟合曲线的目的。

使用的数据在代码里meta_task_data函数中在[-5, 5]范围内随机生成。

## 项目要求

* 构建适用MAML算法的数据集

* 搭建前馈神经网络

* 搭建元学习模型

* 完成训练，探索元学习过程中的特点

## 数据准备

无

## 环境配置安装

无

In [1]:
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
import paddle.io as data
import numpy as np
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


生成 $a*\sin(x+b)$ 的资料点，其中 $a, b$ 的范围分别预设为 $[0.1, 5], [0, 2\pi]$，每一个 $a*\sin(x+b)$ 的函数有 10 个资料点当作训练资料。测试时则可用较密集的资料点直接由画图来看 generalize 的好坏。


In [2]:
device = 'cpu'
def meta_task_data(seed = 0, a_range=[0.1, 5], b_range = [0, 2*np.pi], task_num = 100,
                   n_sample = 10, sample_range = [-5, 5], plot = False):
    np.random.seed(seed)
    a_s = np.random.uniform(low = a_range[0], high = a_range[1], size = task_num)
    b_s = np.random.uniform(low = b_range[0], high = b_range[1], size = task_num)
    total_x = []
    total_y = []
    label = []
    for t in range(task_num):
        x = np.random.uniform(low = sample_range[0], high = sample_range[1], size = n_sample)
        total_x.append(x)
        total_y.append( a_s[t]*np.sin(x+b_s[t]) )
        label.append('{:.3}*sin(x+{:.3})'.format(a_s[t], b_s[t]))
    if plot:
        plot_x = [np.linspace(-5, 5, 1000)]
        plot_y = []
        for t in range(task_num):
            plot_y.append( a_s[t]*np.sin(plot_x+b_s[t]) ) 
        return total_x, total_y, plot_x, plot_y, label
    else:
        return total_x, total_y, label

以下我们将 $\phi$ 称作 meta weight，$\theta$ 则称为 sub weight。

为了让 sub weight 的 gradient 能够传到 meta weight (因为 sub weight 的初始化是从 meta weight 来的，所以想当然我们用 sub weight 算出来的 loss 对 meta weight 也应该是可以算 gradient 才对)，这边我们需要重新定义一些 paddle 内的 layer 的运算。

实际上 *MetaLinear* 这个 class 做的事情跟 paddle.nn.Linear 完全是一样的，唯一的差别在于这边的每一个 tensor 都没有被变成 paddle.nn.Parameter。这么做的原因是因为等一下我们从 meta weight 那里复制 (init weight 输入 meta weight 后 weight 与 bias 使用 .clone) 的时候，tensor 的 clone 的操作是可以传递 gradient 的，以方便我们用 gradient 更新 meta weight。这个写法的代价是我们就没办法使用 paddle.optimizer 更新 sub weight 了，因为参数都只用 tensor 纪录。也因此我们接下来需要自己写 gradient update 的函数 (只用 SGD 的话是简单的)。

In [3]:
class MetaLinear(nn.Layer):
    def __init__(self, init_layer = None):
        super(MetaLinear, self).__init__()
        if type(init_layer) != type(None):
            self.weight = init_layer.weight.clone()
            self.bias = init_layer.bias.clone()
    def clear_grad(self):
        self.weight.grad  = paddle.zeros_like(self.weight)
        self.bias.grad  = paddle.zeros_like(self.bias)
    def forward(self, x):
        return F.linear(x, self.weight, self.bias)

这里的 forward 和一般的 model 是一样的，唯一的差别在于我们需要多写一下 \_\_init\_\_ 函数让他比起一般的 paddle model 多一个可以从 meta weight 复制的功能 (这边因为我把 model 的架构写死了所以可能看起来会有点多余，读者可以自己把 net() 改成可以自己调整架构的样子，然后思考一下如何生成一个跟 meta weight 一样形状的 sub weight)

update 函数就如同前一段提到的，我们需要自己先手动用 SGD 更新一次 sub weight，接着再使用下一步的 gradient (第二步) 来更新 meta weight。clear_grad 函数在此处没有用到，因为实际上我们计算第二步的 gradient 时会需要第一步的 grad，这也是为什么我们第一次 backward 的时候需要 create_graph=True (建立计算图以计算二阶的 gradient)

In [4]:
class net(nn.Layer):
    def __init__(self, init_weight=None):
        super(net, self).__init__()
        if type(init_weight) != type(None):
            for name, module in init_weight.named_sublayers():
                if name != '':
                    setattr(self, name, MetaLinear(module))
        else:
            self.hidden1 = nn.Linear(1, 40)
            self.hidden2 = nn.Linear(40, 40)
            self.out = nn.Linear(40, 1)
    
    def clear_grad(self):
        layers = self.__dict__['_buffers']
        for layer in layers.keys():
            layers[layer].clear_grad()
    def update(self, parent, lr = 1):
        layers = self.__dict__['_buffers']
        parent_layers = parent.__dict__['_buffers']
        for param in layers.keys():
            layers[param].weight = layers[param].weight - lr*parent_layers[param].weight.grad
            layers[param].bias = layers[param].bias - lr*parent_layers[param].bias.grad
        # gradient will flow back due to clone backward
        
    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        return self.out(x)

前面的 class 中我们已经都将复制 meta weight 到 sub weight，以及 sub weight 的更新， gradient 的传递都搞定了，meta weight 自己本身的参数就可以按照一般 paddle model 的模式，使用 paddle.optimizer 来更新了。

gen_model 函数做的事情其实就是产生 N 个 sub weight，并且使用前面我们写好的复制 meta weight 的功能。

注意到复制 weight 其实是整个 code 的关键，因为我们需要将 sub weight 计算的第二步 gradient 正确的传回 meta weight。读者从 meta weight 与 sub weight 更新参数作法的差别 (手动更新 / 用 paddle.nn.Parameter 与 paddle.optimizer) 可以再思考一下两者的差别。



In [5]:
class Meta_learning_model():
    def __init__(self, init_weight = None):
        super(Meta_learning_model, self).__init__()
        self.model = net()
        if type(init_weight) != type(None):
            self.model.load_state_dict(init_weight)
        self.grad_buffer = 0
    def gen_models(self, num, check = True):
        models = [net(init_weight=self.model) for i in range(num)]
        return models
    def clear_buffer(self):
        print("Before grad", self.grad_buffer)
        self.grad_buffer = 0

接下来就是生成训练 / 测试资料，建立 meta weightmeta weight 的模型以及用来比较的 model pretraining 的模型

In [6]:
class TensorDataset(data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        data = self.x[index]
        label = self.y[index]
        return data, label

    def __len__(self):
        return len(self.x)

In [7]:
bsz = 10
train_x, train_y, train_label = meta_task_data(task_num=50000*10) 
train_x = paddle.to_tensor(train_x).unsqueeze(-1) # add one dim
train_y = paddle.to_tensor(train_y).unsqueeze(-1)
train_dataset = TensorDataset(train_x, train_y)
train_loader = data.DataLoader(train_dataset, batch_size=bsz, shuffle=False)

test_x, test_y, plot_x, plot_y, test_label = meta_task_data(task_num=1, n_sample = 10, plot=True)  
test_x = paddle.to_tensor(test_x).unsqueeze(-1) # add one dim
test_y = paddle.to_tensor(test_y).unsqueeze(-1) # add one dim
plot_x = paddle.to_tensor(plot_x).unsqueeze(-1) # add one dim
test_dataset = TensorDataset(test_x, test_y)
test_loader = data.DataLoader(test_dataset, batch_size=bsz, shuffle=False)  

meta_model = Meta_learning_model()

meta_optimizer = paddle.optimizer.Adam(parameters=meta_model.model.parameters(), learning_rate=1e-3)


pretrain = net()
pretrain.train()
pretrain_optim = paddle.optimizer.Adam(parameters=pretrain.parameters(), learning_rate=1e-3)

进行训练，注意一开始我们要先生成一群 sub weight (code 里面的 sub models)，然后将一个 batch 的不同的 sin 函数的 10 笔资料点拿来训练 sub weight。注意这边 sub weight 计算第一步 gradient 与第二步 gradient 时使用各五笔不重复的资料点 (因此使用 [:5] 与 [5:] 来取)。但在训练 model pretraining 的对照组时则没有这个问题 (所以 pretraining 的 model 是可以确实的走两步 gradient 的)

每一个 sub weight 计算完 loss 后相加 (内层的 for 回圈) 后就可以使用 optimizer 来更新 meta weight，再次提醒一下 sub weight 计算第一次 loss 的时候 backward 是需要 create_graph=True 的，这样计算第二步 gradient 的时候才会真的计算到二阶的项。读者可以在这个地方思考一下如何将这段程式码改成 MAML 的一阶做法。

温馨提示: 这个训练耗时极久请耐心等待.

In [ ]:
epoch = 1
for e in range(epoch):
    meta_model.model.train()
    for x, y in tqdm(train_loader):
        sub_models = meta_model.gen_models(bsz)
        meta_l = 0
        for model_num in range(len(sub_models)):
            sample = list(range(10))
            np.random.shuffle(sample)
            
            #pretraining
            pretrain_optim.clear_grad()
            index_x = paddle.to_tensor(sample[:5])
            x1 = paddle.index_select(x=x[model_num], index=index_x)
            y_tilde = pretrain(x1)

            index_y = paddle.to_tensor(sample[:5])
            y1 = paddle.index_select(x=y[model_num], index=index_y)

            little_l = paddle.nn.MSELoss()
            loss = little_l(y_tilde, y1)
            
            loss.backward()
            pretrain_optim.step()
            pretrain_optim.clear_grad()

            index_x2 = paddle.to_tensor(sample[5:])
            x2 = paddle.index_select(x=x[model_num], index=index_x2)

            y_tilde = pretrain(x2)

            index_y2 = paddle.to_tensor(sample[5:])
            y2 = paddle.index_select(x=y[model_num], index=index_y2)

            loss_2 = little_l(y_tilde, y2)
            loss_2.backward()
            pretrain_optim.step()
        
            # meta learning
            
            y_tilde = sub_models[model_num](x1)
            little_l = F.mse_loss(y_tilde, y1)
            #计算第一次 gradient 并保留计算图以接着计算更高阶的 gradient
            little_l.backward(retain_graph=True)
            # little_l.backward(create_graph = True)
            sub_models[model_num].update(lr = 1e-2, parent = meta_model.model)
            #先清空 optimizer 中计算的 gradient 值 (避免累加)
            meta_optimizer.clear_grad()
            
            #计算第二次 (二阶) 的 gradient，二阶的原因来自第一次 update 时有计算过一次 gradient 了
            y_tilde = sub_models[model_num](x2)
            meta_l =  meta_l + F.mse_loss(y_tilde, y2)

        meta_l = meta_l / bsz
        meta_l.backward()
        meta_optimizer.step()
        meta_optimizer.clear_grad()

  0%|          | 93/50000 [00:06<54:11, 15.35it/s]  

测试我们训练好的 meta weight

In [ ]:
# test_model = copy.deepcopy(meta_model.model)
test_model = meta_model.model
test_model.train()
test_optim = paddle.optimizer.SGD(parameters=test_model.parameters(), learning_rate=1e-3)

先画出待测试的 sin 函数，以及用圆点点出测试时给 meta weight 训练的十笔资料点

In [ ]:
fig = plt.figure(figsize = [9.6,7.2])
ax = plt.subplot(111)
plot_x1 = plot_x.squeeze().numpy()
ax.scatter(test_x.numpy().squeeze(), test_y.numpy().squeeze())
ax.plot(plot_x1, plot_y[0].squeeze())

分别利用十笔资料点更新 meta weight 以及 pretrained model 一个 step

In [ ]:
test_model.train()
pretrain.train()

for epoch in range(1):
    for x, y in test_loader:
        y_tilde = test_model(x[0])
        little_l = paddle.nn.MSELoss()
        loss = little_l(y_tilde, y[0])
        test_optim.clear_grad()
        loss.backward()
        test_optim.step()
        print("(meta)))Loss: ", loss)

for epoch in range(1):
    for x, y in test_loader:
        y_tilde = pretrain(x[0])
        little_l = paddle.nn.MSELoss()
        loss = little_l(y_tilde, y[0])
        pretrain_optim.clear_grad()
        loss.backward()
        pretrain_optim.step()
        print("(pretrain)Loss: ", loss)

將更新后的模型所代表的函数绘制出來，与真实的 sin 函数比較

In [ ]:
test_model.eval()
pretrain.eval()

plot_y_tilde = test_model(plot_x[0]).squeeze().detach().numpy()
plot_x2 = plot_x.squeeze().numpy()
ax.plot(plot_x2, plot_y_tilde, label = 'tune(disjoint)')
ax.legend()
fig.show()

In [ ]:
plot_y_tilde = pretrain(plot_x[0]).squeeze().detach().numpy()
plot_x2 = plot_x.squeeze().numpy()
ax.plot(plot_x2, plot_y_tilde, label = 'pretrain')
ax.legend()

执行底下的 cell 以显示图形，并重复执行更新 meta weight 与 pretrained model 的 cell 来比较多更新几步后是否真的能看出 meta learning 比 model pretraining 有效

In [ ]:
fig